In [1]:
import numpy as np
from scipy.stats import norm, t
import math
import pandas as pd
import matplotlib.pyplot as plt
import os
import matplotlib
from tqdm import tqdm
from scipy.stats import gmean

In [2]:
S_0 = 100 # initial price
K = 100 # Strike Price
r = 0.01 # risk-free rate
sigma = 0.1 # Vol
T = 1 # time to maturity
N = 260 # number of time step
n = 10**5 # simulation time
n_pil = 100

In [3]:
# Q5f for (b)
def Naive_MonteCarlo(K,S_0,n):

    res = []

    # simulation
    for i in tqdm(range(n)):
        
        # price path
            
        prices = [0] * (N+1)
        # initial price is S_0
        prices[0] = S_0
        dt = T/N # time period
        
        for t_i in range(1,N+1,1):
            
            # time interval
            Z = np.random.normal(0, np.sqrt(dt))
            prices[t_i] = prices[t_i-1]* np.exp((r-0.5*sigma**2)*dt+sigma*Z)

        curavg = np.mean(prices)

        payoff = max(curavg-K,0)
        fair_price = payoff* np.exp(0-r*T)
        res.append(fair_price)

    # calculate the confidence interval
        
    mu_mc = np.mean(res)
    
    # unbaised standard deviation:
    sum_of_square = 0
    for realized in res:
        sum_of_square += (realized-mu_mc)**2
    sample_std = np.sqrt(sum_of_square/(len(res)-1))
    
    # Z-score for 95% confidence
    z_score = norm.ppf(.975)

    # Print the results
    print(f"95% Confidence Interval: ({mu_mc-z_score*sample_std/np.sqrt(len(res))}, {mu_mc+z_score*sample_std/np.sqrt(len(res))})")
    # Confidence interval
    return [mu_mc-z_score*sample_std/np.sqrt(len(res)),
            mu_mc+z_score*sample_std/np.sqrt(len(res))]


        
    
        

In [ ]:
# Q5f for (c)
def AV_montecarlo(K,S_0,n,N):
    
    res = []
    res_AV=[]
    # simulation
    for i in tqdm(range(n//2)):
        
        # price path
            
        prices = [0] * (N+1)
        # initial price is S_0
        prices[0] = S_0
        dt = T/N # time period
        
        antithetic_prices = [0]*(N+1)
        # initial price is S_0
        antithetic_prices[0] = S_0
        for t_i in range(1,N+1,1):
            
            # time interval
            Z = np.random.normal(0, np.sqrt(dt))
        
            prices[t_i] = prices[t_i-1]* np.exp((r-0.5*sigma**2)*dt+sigma*Z)

#           #Antithetic price
            antithetic_prices[t_i] = antithetic_prices[t_i-1]* np.exp((r-0.5*sigma**2)*dt-sigma*Z)
        curavg = np.mean(prices)
        curavg_antithetic = np.mean(antithetic_prices)

        payoff = max(curavg-K,0)
        payoff_antithetic = max(curavg_antithetic-K,0)

        fair_price = payoff* np.exp(0-r*T)
        fair_price_antithetic = payoff_antithetic*np.exp(0-r*T)
       
        res.append(fair_price)
        res_AV.append(fair_price_antithetic)


    # Z-score for 95% confidence
    z_score = norm.ppf(.975)
    
    ## calculate the mu AV
    mu_av = np.mean((np.array(res)+np.array(res_AV))/2)

    ## calcuilate the sample Variance
    sumofsquare = 0
    for i in range(len(res)):
        sumofsquare += ((res[i]+res_AV[i])/2-mu_av)**2

    samplestd = np.sqrt(sumofsquare/(len(res)-1))
    print(len(res))
    print(len(res_AV))
        
    # Print the results
    print(f"95% Confidence Interval: ({mu_av-z_score*samplestd/np.sqrt(len(res)/2)}, {mu_av+z_score*samplestd/np.sqrt(len(res)/2)})")
    # Confidence interval
    return [mu_av-z_score*samplestd/np.sqrt(len(res)/2),
            mu_av+z_score*samplestd/np.sqrt(len(res)/2)]
    

In [ ]:
# Q5f for (e)

def CV_AsianCall(K,S_0,N,n_pil,n):

        # calculate beta first using pilot run
    res_pil = []
    res_CV_pil=[]
    # pilot run
    for i in tqdm(range(n_pil)):

        # price path

        prices = [0] * (N+1)
        # initial price is S_0
        prices[0] = S_0
        dt = T/N # time period

        #CV_prices = [0]* (N+1)
        # initial price is S_0
        #CV_prices[0] = S_0
        for t_i in range(1,N+1,1):

            # time interval
            Z = np.random.normal(0, np.sqrt(dt))

            prices[t_i] = prices[t_i-1]* np.exp((r-0.5*sigma**2)*dt+sigma*Z)

    #          
            #CV_prices[t_i] = CV_prices[t_i-1]* np.exp((r-0.5*sigma**2)*dt+sigma*Z)
        curavg = np.mean(prices) # Y
        geo_mean = gmean(prices) # control variable
        #print(prices)

        res_pil.append(max(0,curavg-K)* np.exp(0-r*T))
        res_CV_pil.append(max(0,geo_mean-K)* np.exp(0-r*T))

    mu_pil = np.mean(res_pil)
    mu_CV_pil = np.mean(res_CV_pil)
    #sample deviation
    sum_of_square = 0
    for realized in res_CV_pil:
        sum_of_square += (realized-mu_CV_pil)**2
    sigma_C_pil = np.sqrt(sum_of_square/(n_pil-1))
    # sample beta
    beta_hat = ((np.inner(np.array(res_pil), np.array(res_CV_pil)))-n_pil*(mu_pil*mu_CV_pil))/((n_pil-1)*(sigma_C_pil**2))

    print("the internal beta is: {}".format(beta_hat))
# now the actual simulation



    res = []
    res_CV=[]
    for i in tqdm(range(n)):

        # price path

        prices = [0] * (N+1)
        # initial price is S_0
        prices[0] = S_0
        dt = T/N # time period

        #CV_prices = [0]* (N+1)
        # initial price is S_0
        #CV_prices[0] = S_0
        for t_i in range(1,N+1,1):

            # time interval
            Z = np.random.normal(0, np.sqrt(dt))

            prices[t_i] = prices[t_i-1]* np.exp((r-0.5*sigma**2)*dt+sigma*Z)

    #          
            #CV_prices[t_i] = CV_prices[t_i-1]* np.exp((r-0.5*sigma**2)*dt+sigma*Z)
        curavg = np.mean(prices) # Y
        geo_mean = gmean(prices) # control variable
        #print(prices)

        res.append(max(0,curavg-K)* np.exp(0-r*T))
        res_CV.append(max(0,geo_mean-K)* np.exp(0-r*T))
    
    # mu_c for distributiin
    a = np.log(S_0) + (r - sigma**2 * 0.5) * T * (N + 1) / (2 * N)
    b = sigma**2 * T * (N + 1) * (2 * N + 1) / (6 * N**2)

    # Calculate d1 and d2 using a and b
    d1 = (-np.log(K) + a + b) / np.sqrt(b)
    d2 = d1 - np.sqrt(b)

    # Calculate mu_c using the cumulative distribution function of a standard normal
    mu_c = np.exp(-r * T) * (np.exp(a + b/2) * norm.cdf(d1) - K * norm.cdf(d2))

    mu_n_CV = 0
    for i in range(n):
        mu_n_CV += (res[i]+beta_hat*(mu_c-res_CV[i]))
    mu_n_CV = mu_n_CV/n

    # now calculate the sample standard deviation
    #sample deviation
    sum_of_square = 0
    for realized in res_CV:
        sum_of_square += (realized-mu_n_CV)**2
    sigma_hat = np.sqrt(sum_of_square/(n*(n-1)))

    # Z-score for 95% confidence
    z_score = norm.ppf(.975)
    # Print the results
    print(f"95% Confidence Interval: ({mu_n_CV-z_score*sigma_hat/np.sqrt(len(res))},{mu_n_CV+z_score*sigma_hat/np.sqrt(len(res))})")
    # Confidence interval
#return [mu_n_CV-z_score*sigma_hat/np.sqrt(len(res)),
#        mu_n_CV-z_score*sigma_hat/np.sqrt(len(res))]

In [ ]:
# Q5h for (g)
def Binomial_AsianCall(K,S_0,N):
    
    r=0.01 # risk free rate
    sigma = 0.1 # Val
    T = 1
    u = np.exp(sigma*np.sqrt(T/N))
    d = 1/u
    dt = T/N
    q_u=1/2
    q_d=1-q_u
    print("u is {}".format(u))
    print("d is {}".format(d))
    # simulate the prices at T.

    price_paths = []
    # generate the price path first, also calculate the number of upper in order 
    # in order to 
    def helper(current_path,n,price,):
        
        if n == N:
            
            price_paths.append(current_path)
            
            return

        else:
            helper(current_path+[price*u],n+1,price*u)
            helper(current_path+[price*d],n+1, price*d)

    helper([S_0],0,S_0)     


    # Now calculate the payoff at time T first.
    V_N = []

    for price_path in price_paths:
        payoff = max(0,np.mean(price_path)-K)
        V_N.append(payoff)

    # now using backward induction to calculate the V_0 price
    #print(V_N)
    for t in range(N-1,-1,-1):
        #print(t)
        # payoff list at time t.  backward induction from every two up and down price
        if t == N-1:
            V_t = V_N
        Pre = []
        i = 0
        while i <= len(V_t)-1:
            
            Pre_payoff = np.exp(0-r*dt)*(q_u*V_t[i]+q_d*V_t[i+1])
            Pre.append(Pre_payoff)

            i = i+2

        #print(Pre)
        V_t = Pre
    print('For N = {}, V_0 value is {}'.format(N,V_t))
    return V_t



In [ ]:
 # (b)
Naive_MonteCarlo(K=110,S_0=100,n=100000,N=260)



100%|██████████| 100000/100000 [01:24<00:00, 1181.69it/s]


95% Confidence Interval: (0.15567418065967284, 0.16677638549544974)


[0.15567418065967284, 0.16677638549544974]

In [ ]:
#(c)
AV_montecarlo(K=110,S_0=100,n=100000,N=260)

100%|██████████| 50000/50000 [01:00<00:00, 829.76it/s]

50000
50000
95% Confidence Interval: (0.15153121920964693, 0.16710018547096953)


[0.15153121920964693, 0.16710018547096953]

In [ ]:
#(e)
CV_AsianCall(K=110,S_0=100,N=260,n_pil=100,n=100000)

100%|██████████| 100/100 [00:00<00:00, 944.65it/s]


the internal beta is: 1.046017076497132


100%|██████████| 100000/100000 [01:46<00:00, 935.50it/s]

95% Confidence Interval: (0.1615499582042672,0.16158302279737613)


# Comment: 
the above are the three confidence interver. The control variable method has the narriest confidence interval 
#and the least variance

In [ ]:
# code for (g) 
for timeinterval in [5,10,20]:
    Binomial_AsianCall(K=110,S_0=100,N=timeinterval)

u is 1.0457364348384068
d is 0.9562638985171495
For N = 5, V_0 value is [0.07821921495423305]
u is 1.0321280889960358
d is 0.9688719943400753
For N = 10, V_0 value is [0.1106669775014856]
u is 1.0226125536284048
d is 0.9778874672052759
For N = 20, V_0 value is [0.12694444857302675]


In [5]:
res = []

# simulation
for i in tqdm(range(100000)):

    # price path            
    prices = [0] * (N+1)
    # initial price is S_0
    prices[0] = S_0
    dt = T/N # time period

    for t_i in range(1,N+1,1):

        # time interval
        Z = np.random.normal(0, np.sqrt(dt))
        prices[t_i] = prices[t_i-1]* np.exp((r-0.5*sigma**2)*dt+sigma*Z)

    curavg = np.mean(prices)

    payoff = max(curavg-K,0)
    fair_price = payoff* np.exp(0-r*T)
    res.append(fair_price)

# calculate the confidence interval

mu_mc = np.mean(res)

# unbaised standard deviation:
sum_of_square = 0
for realized in res:
    sum_of_square += (realized-mu_mc)**2
sample_std = np.sqrt(sum_of_square/(len(res)-1))

# Z-score for 95% confidence
z_score = norm.ppf(.975)

# Print the results
print(f"95% Confidence Interval: ({mu_mc-z_score*sample_std/np.sqrt(len(res))}, {mu_mc+z_score*sample_std/np.sqrt(len(res))})")
# Confidence interval

100%|██████████| 100000/100000 [01:29<00:00, 1121.20it/s]


95% Confidence Interval: (2.5150450636579973, 2.560328872268951)


In [7]:
sample_std

3.653127747940842

In [9]:
np.std(res)

3.6531094822555725